# Guide for UPBIT Quotation API

In [2]:
# setting for path
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('')))

# import libraries
import numpy as np
import pandas as pd
import datetime
from utils.upbit_api import UpbitQuotationAPI

In [3]:
# call upbit quotation api
upbit = UpbitQuotationAPI()


### 예시 : 1분 단위로 하루치(1440m) 데이터 가져오기
2022.12.25 09:00:00까지 1440분(즉 24시간)의 KRW-BTC (원화시장 비트코인) 데이터를 받아옴


즉, 2022.12.24 09:00 부터, 2022.12.25 08:59:00 까지

In [4]:
to_ = datetime.datetime(2022, 12, 25, 9, 0, 0)
df = upbit.get_1min_ohlcv(market="KRW-BTC", to=to_, count=1440)
df

,open,high,low,close,volume,value
2022-12-24 17:38:00,21932000.0,21938000.0,21931000.0,21938000.0,0.348331,7.639690e+06
2022-12-24 17:39:00,21940000.0,21953000.0,21940000.0,21953000.0,0.485640,1.065979e+07
2022-12-24 17:40:00,21953000.0,21957000.0,21941000.0,21957000.0,0.316569,6.949239e+06
2022-12-24 17:41:00,21957000.0,21957000.0,21940000.0,21941000.0,0.816832,1.792473e+07
2022-12-24 17:42:00,21943000.0,21943000.0,21931000.0,21931000.0,0.815186,1.788232e+07
...,...,...,...,...,...,...
2022-12-25 17:55:00,21891000.0,21891000.0,21879000.0,21880000.0,4.242986,9.285848e+07
2022-12-25 17:56:00,21880000.0,21891000.0,21878000.0,21878000.0,1.785182,3.905992e+07
2022-12-25 17:57:00,21880000.0,21880000.0,21870000.0,21871000.0,6.454913,1.411883e+08
2022-12-25 17:58:00,21871000.0,21891000.0,21871000.0,21880000.0,2.924536,6.399381e+07


단, 주의해야할 사항이 하나 있다. 

에러 방지를 위해, count의 최대 개수를 2000으로 제한해두었다.

2000이상을 호출할 때는 get_1min_ohlcb 함수를 여러번 분할해서 사용하는 것을 추천한다.

아래의 예시를 살펴보자.

In [5]:
dfs = [] # 여러 dataframe을 임시로 담아둘 list

# 12월 20~29일을 to로 삼아서 dataframe을 UPBIT로부터 호출해보자.
for ith_day in range (20, 30): 
    to_ = datetime.datetime(2022, 12, ith_day, 9, 0, 0)
    df_ = upbit.get_1min_ohlcv(market="KRW-ETH", to=to_, count=1440)
    dfs += [df_]

final_df = pd.concat(dfs).sort_index() # 합치기
final_df

,open,high,low,close,volume,value
2022-12-19 17:51:00,1561000.0,1561000.0,1560500.0,1561000.0,43.329188,6.763544e+07
2022-12-19 17:52:00,1561500.0,1562000.0,1561500.0,1561500.0,52.354121,8.177189e+07
2022-12-19 17:53:00,1561500.0,1562500.0,1561500.0,1562500.0,21.182309,3.309510e+07
2022-12-19 17:54:00,1562500.0,1563000.0,1562000.0,1562500.0,85.928725,1.343023e+08
2022-12-19 17:55:00,1562000.0,1562500.0,1561500.0,1562000.0,28.750314,4.491370e+07
...,...,...,...,...,...,...
2022-12-29 17:55:00,1515500.0,1515500.0,1515500.0,1515500.0,9.777874,1.481837e+07
2022-12-29 17:56:00,1515500.0,1516000.0,1515500.0,1515500.0,6.003776,9.099060e+06
2022-12-29 17:57:00,1515500.0,1516000.0,1515500.0,1515500.0,10.040995,1.522198e+07
2022-12-29 17:58:00,1515500.0,1516000.0,1515500.0,1515500.0,4.145838,6.283639e+06


그런데.... 사실 방금 해보면서 느낀건데, for문을 통해 처리하기가 조금 귀찮다...

추후에 upbit quoutation api class 안에, 조금 더 편하게 불러올 수 있는 함수들을 추가해놓겠다.

### Pandas Dataframe 저장하고, 불러오기

In [9]:
# 위에서 불러온 14400 줄짜리, dataframe final_df를 hdf5 형식으로 저장해보자.
final_df.to_hdf("data/test_save.h5", key="df", mode="w")

# 혹시 위 코드에서 에러가 나면, [pip install tables] 명령어를 통해 pytables를 설치해보세요~

In [10]:
# 이제 저장된 test_save.h5를 불러보자.
new_df = pd.read_hdf("data/test_save.h5", "df")
new_df

,open,high,low,close,volume,value
2022-12-19 17:51:00,1561000.0,1561000.0,1560500.0,1561000.0,43.329188,6.763544e+07
2022-12-19 17:52:00,1561500.0,1562000.0,1561500.0,1561500.0,52.354121,8.177189e+07
2022-12-19 17:53:00,1561500.0,1562500.0,1561500.0,1562500.0,21.182309,3.309510e+07
2022-12-19 17:54:00,1562500.0,1563000.0,1562000.0,1562500.0,85.928725,1.343023e+08
2022-12-19 17:55:00,1562000.0,1562500.0,1561500.0,1562000.0,28.750314,4.491370e+07
...,...,...,...,...,...,...
2022-12-29 17:55:00,1515500.0,1515500.0,1515500.0,1515500.0,9.777874,1.481837e+07
2022-12-29 17:56:00,1515500.0,1516000.0,1515500.0,1515500.0,6.003776,9.099060e+06
2022-12-29 17:57:00,1515500.0,1516000.0,1515500.0,1515500.0,10.040995,1.522198e+07
2022-12-29 17:58:00,1515500.0,1516000.0,1515500.0,1515500.0,4.145838,6.283639e+06


### 예시: 실시간 데이터 가져오기

In [11]:
btc = upbit.get_live_price(market="KRW-BTC") # 비트코인
eth = upbit.get_live_price(market="KRW-ETH") # 이더리움
print(btc)
print(eth)


(29106000.0, datetime.datetime(2023, 1, 30, 15, 16, 39), datetime.datetime(2023, 1, 30, 15, 16, 41, 453626, tzinfo=<UTC>))
(1992000.0, datetime.datetime(2023, 1, 30, 15, 16, 31), datetime.datetime(2023, 1, 30, 15, 16, 41, 506598, tzinfo=<UTC>))


get_live_price 함수를 호출하면, (가격, 최근 거래일시, 함수 호출 일시)가 반환된다.

위의 코드를 실행했을때, <최근 거래일시>가 아래와 같았다.

btc -> datetime.datetime(2023, 1, 30, 15, 16, 39)

eth -> datetime.datetime(2023, 1, 30, 15, 16, 31)

그리고 함수 호출 일시는 datetime.datetime(2023, 1, 30, 15, 16, 41) 였다.

즉, 여러 마켓의 최근거래일시가 꼭 일치한다는 법도 없고, 함수 호출 일시와 다를수도 있다.

(왜냐하면 업비트 거래소에서 해당 코인에 대해 매초마다 거래가 이뤄지지 않을 수도 있기 때문이다.)

따라서, 우리가 여러 마켓에 대해 가격 정보를 갖고 오고 싶을때, 

<함수 호출 일시>로 통일을 해야, 시간대를 통일시킬수 있을 것이다.


In [12]:
lst = [[btc[0], eth[0]]]
df = pd.DataFrame(lst, columns=["KRW-BTC", "KRW-ETH"],  dtype=float)
df.index = [btc[2].strftime("%Y-%m-%d %H:%M")]

df

,KRW-BTC,KRW-ETH
2023-01-30 15:16,29106000.0,1992000.0


### 예시: 실시간 데이터 가져오기 (마치 실전처럼 1분 간격으로 3개를 가져와서 합치기)

In [13]:
# 이 코드 블럭은 실행하는데 약 3분 걸림
import time

dfs = []

for i in range(0, 3):
    btc = upbit.get_live_price(market="KRW-BTC") # 비트코인
    eth = upbit.get_live_price(market="KRW-ETH") # 이더리움
    time.sleep(60) # 1분(60s) 동안 코드 정지 시키기
    lst = [[btc[0], eth[0]]]
    df = pd.DataFrame(lst, columns=["KRW-BTC", "KRW-ETH"],  dtype=float)
    df.index = [btc[2].strftime("%Y-%m-%d %H:%M")]
    dfs += [df]

df_3min = pd.concat(dfs).sort_index()
df_3min




,KRW-BTC,KRW-ETH
2023-01-30 15:16,29106000.0,1992000.0
2023-01-30 15:17,29101000.0,1991000.0
2023-01-30 15:18,29105000.0,1991000.0


그런데.... 사실 방금 해보면서 또 느낀건데, for문을 통해 처리하기가 조금 귀찮다...

추후에 upbit quoutation api class 안에, 조금 더 편하게 불러올 수 있는 함수들을 추가해놓겠다.